In [ ]:
# Pendientes
# Reindex
# Remove values during field campaigns
# Repetir para los pozos del sitio sur

# Estrategia: reindexar los datasets a escala horaria, corrigiendo duplicados y rellenando con NaNs.
#             Luego, generar series temporales a escala diaria para uso de todos en el proyecto.

In [104]:
# Importar las bibliotecas necesarias
import pandas as pd

In [105]:
# Definir la ruta de los datos

# Sitio norte
fp_sdh1_ps01 = 'data/raw/piezometers/Data_15_07_2025_ SDH1PS01_COMPENSADA.xlsx'
fp_sdh1_ps02 = 'data/raw/piezometers/Data_15_07_2025_ SDH1PS02_COMPENSADA.xlsx'

In [106]:
# Leer los datos

# Sitio norte
sdh1_ps01 = pd.read_excel(fp_sdh1_ps01)
sdh1_ps02 = pd.read_excel(fp_sdh1_ps02)

In [107]:
# Función para crear una columna 'Timestamps' y usarla como indice con formato datetime

def timestamp_as_index(dataframe):

    # Convertir las columnas 'Date' y 'Time' en strings
    date_str = dataframe['Date'].astype(str)
    time_str = dataframe['Time'].astype(str)

    # Crear columna 'Timestamps' con formato datetime
    dataframe['Timestamps'] = pd.to_datetime(
        date_str + ' ' + time_str,
        format='%Y-%m-%d %H:%M:%S'
    )

    # Establecer 'Timestamps' como indice
    dataframe = dataframe.set_index('Timestamps')
    
    # Eliminar columnas 'Date' y 'Time'
    dataframe = dataframe.drop(columns=['Date', 'Time'])

    return dataframe

In [108]:
# Ejecutar funcion para establecer 'Timestamps' como indice
sdh1_ps01 = timestamp_as_index(sdh1_ps01)
sdh1_ps02 = timestamp_as_index(sdh1_ps02)

In [109]:
# Diccionario con nombres de columnas originales y nuevas

columns_dict = {
    'TEMPERATURE' : 'Temperature_C',
    'NE_m' : 'Depth_m',
    'Cota_m' : 'Static_level_masl'
}

# Lista con nombres de columnas no necesarias
columns_list = [
    'ms',
    'LEVEL',
    'P_baro'
]

In [110]:
# Funcion para renombrar columnas y eliminar las innecesarias

def format_columns(df):
    df = df.rename(columns=columns_dict).drop(columns=columns_list, axis=1, errors='ignore')
    return df

In [111]:
# Ejecutar la función de formateo de columnas

sdh1_ps01 = format_columns(sdh1_ps01)
sdh1_ps02 = format_columns(sdh1_ps02)

In [ ]:
# Función para detectar datos duplicados
def check_duplicates(df, df_name):

    # Hace una copia del df
    df_copy = df.copy()

    # Genera una columna que almacena los registros duplicados como True
    df_copy['duplicated'] = df_copy.index.duplicated(keep=False)

    # Filtra la copia del df, manteniendo sólo los registros duplicados
    dup_filter = df_copy[df_copy['duplicated'] == True]

    # Si los hay, imprime cuántos y cuáles registros están duplicados
    if dup_filter.empty:
        print(f'No hay datos duplicados en {df_name}')
    else:
        print(f'{df_name} tiene {len(dup_filter)} datos duplicados:')
        print(dup_filter)

In [117]:
# Aplicar función de detección de duplicados
check_duplicates(sdh1_ps01, 'sdh_ps01')
check_duplicates(sdh1_ps02, 'sdh_ps02')

No hay datos duplicados en sdh_ps01
No hay datos duplicados en sdh_ps02


In [ ]:
# Función para detectar saltos en los datos
# Hay que identificar los periodos de 5, 10 y 15 min de registros y adaptar la función para cada uno

In [119]:
sdh1_ps01

,Temperature_C,Depth_m,Static_level_masl
Timestamps,,,
2024-05-23 00:00:00,9.801,0.5969,3827.0031
2024-05-23 00:05:00,9.802,0.5966,3827.0034
2024-05-23 00:10:00,9.797,0.5971,3827.0029
2024-05-23 00:15:00,9.796,0.5970,3827.0030
2024-05-23 00:20:00,9.796,0.5969,3827.0031
...,...,...,...
2025-07-15 09:30:00,5.902,0.5339,3827.0661
2025-07-15 09:40:00,5.901,0.5350,3827.0650
2025-07-15 09:50:00,5.899,0.5348,3827.0652
